In [ ]:
%pip install spotipy

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import pandas as pd
import time
import datetime
import os

client_id=' 공백 '
client_secret=' 공백 '


client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
import time
import datetime
import pandas as pd

start_time = time.time()

year_list = ['2023', '2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010','2009', '2008', '2007', '2006', '2005', '2004', '2003', '2002', '2001', '2000']
genre_list = ['k-pop', 'acoustic', 'club', 'dance', 'edm', 'disco', 'electronic', 'happy', 'hip-hop','party', 'pop', 'r-n-b', 'rock', 'romance', 'sleep', 'soul']


total_workload = len(year_list) * len(genre_list) * 100
completed_workload = 0

for genre in genre_list:
    for year in year_list:
        print(f"{year}_{genre}: 다운로드")

        track_features_list = []

        count = 1
        start_time = time.time()

        for i in range(0, 1000, 50):
            track_results = sp.search(q=f'year:{year} genre:{genre}', type='track', limit=50, offset=i)
            for j, t in enumerate(track_results['tracks']['items']):
                print(f"{year}_{genre}: 진행중")

                track_features = sp.audio_features([t['id']])
                if track_features:
                    track_features_list.append(track_features[0])
                else:
                    track_features_list.append(None)

                completed_workload += 1
                progress = (completed_workload / total_workload) * 100
                print(f"전체 진행 상황 : {count}개 ... 예상{progress/10:.2f}%")
                #time.sleep(3)

                count += 1

        track_df = pd.DataFrame(track_features_list)
        track_df.drop_duplicates(subset='id', inplace=True)
        track_df = track_df.rename(columns={'id': 'track_id'})
        track_df = track_df.rename(columns={'key': 'music_key'})
        track_df = track_df.rename(columns={'mode': 'music_mode'})
        track_df = track_df.drop(columns=['type', 'uri', 'track_href', 'analysis_url', 'duration_ms'])

        timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
        #filename = f'./data/{genre}_audio_{year}_{timestamp}.csv'
        filename = f'/home/ubuntu/project/4.5HZ/Data/test/API/audio/{genre}_audio_{year}_{timestamp}.csv'

        track_df.to_csv(filename, index=False)

        end_time = time.time()
        intermediate_time = end_time - start_time
        intermediate_time_formatted = str(datetime.timedelta(seconds=int(intermediate_time)))
        print(f"중간 작업 소요 시간: {intermediate_time_formatted}")
        print(f" 다음 작업 대기")
        time.sleep(940)

end_time = time.time()
total_time = end_time - start_time
total_time_formatted = str(datetime.timedelta(seconds=int(total_time)))
print(f"총 작업 소요 시간: {total_time_formatted}")
